In [1]:
#!pip install pycombo
#!pip install fastnode2vec

In [3]:
#import pycombo
#import fastnode2vec

In [2]:
import networkx as nx
import numpy as np
import time
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

from keras.utils import to_categorical
from sklearn.mixture import GaussianMixture

/Users/stanislav/opt/anaconda3/envs/Python37pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def loadNetworkMat(filename, path = '/Users/stanislav/Desktop/NYU/NYURESEARCH/STRATEGIC_RESEARCH/ModularityMaximum/SampleNetworks/ProcessedMat/'):
    A = scipy.io.loadmat(path + filename)
    if check_symmetric(A['net']):
        G = nx.from_numpy_matrix(A['net'])
    else:
        G = nx.from_numpy_matrix(A['net'], create_using=nx.DiGraph)
    return G

In [4]:
def modularity_matrix(adj):
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(Q, partition):
    return (Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()

def residential_page_rank_embedding(A, alpha=0.85):
    w = A.sum(axis = 1).reshape(-1,1)
    n = A.shape[0]
    A = (A + (w == 0)) / (w + n * (w == 0))
    AI = np.linalg.inv(np.eye(n) - A * alpha)
    X = (1 - alpha) * AI
    return X

def node2vec_embedding(G: nx.Graph, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42):
    if nx.is_weighted(G):
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1]), edge[2]['weight']) for edge in G.edges(data=True)],
                directed=False, weighted=True)
    else:
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1])) for edge in G.edges(data=True)],
                    directed=False, weighted=False)
    n2v = fastnode2vec.Node2Vec(n2v_graph, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    n2v.train(epochs=100)
    n2v_embeddings = np.array([n2v.wv[str(node)] for node in G])
    return n2v_embeddings

def rpr_clustering(A: np.array, n_clusters=4, kmeans_runs=100, alpha=0.85, seed=42):
    rpr_embedding = residential_page_rank_embedding(A, alpha)
    X = StandardScaler().fit_transform(X=rpr_embedding)
    rpr_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(X).labels_
    return rpr_cluster_labels

def n2v_clustering(G: nx.Graph, n_clusters=4, kmeans_runs=100, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42):
    n2v_embeddings = node2vec_embedding(G, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    X = StandardScaler().fit_transform(X=n2v_embeddings)
    n2v_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(X).labels_
    return n2v_cluster_labels

In [5]:
def aggregateEmbedding(X, n_groups):
  alpha = 0.85
  kmeans_runs = 100
  seed = 1
  XS = StandardScaler().fit_transform(X = X)
  model = KMeans(n_clusters = n_groups, n_init=kmeans_runs, random_state=seed).fit(XS)
  #model = GaussianMixture(n_components = n_groups, n_init=kmeans_runs, random_state=seed).fit(X)
  #rpr_cluster_labels = model.predict_proba(X)
  cluster_labels = model.predict(XS)
  XC = model.cluster_centers_
  dist2 = np.array([[((XS[i, :] - XC[j, :])**2).sum() for j in range(n_groups)] for i in range(X.shape[0])])
  sigma2 = dist2.min(axis = 1).mean()
  probs = np.exp(- dist2 / sigma2)
  probs /= probs.sum(axis = 1).reshape(-1,1)
  XA = np.matmul(X, probs)
  return XA

In [6]:
def modularity_matrix_np(adj):
    if not(isinstance(adj, np.ndarray)):
        adj = nx.to_numpy_array(G)
    w_in = adj.sum(axis=0).reshape(-1,1)
    w_out = adj.sum(axis=1).reshape(1,-1)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

In [9]:
def clusterEmbeddings(X, n_comms = 2, cluststyle = 'k-means', seed = 1, runs = 100):
    if cluststyle == 'GM':
        XS = StandardScaler().fit_transform(X = X)
        model = GaussianMixture(n_components = n_comms, n_init=runs, random_state=seed).fit(XS)
        partition = model.predict(XS)
    else:
        XS = StandardScaler().fit_transform(X = X)
        model = KMeans(n_clusters=n_comms, n_init=runs, random_state=seed).fit(XS)
        partition = model.labels_
    return partition, model

In [10]:
def produceAggEmbed(A, n_agg, alpha = 0.85):
    X = residential_page_rank_embedding(A, alpha = alpha)
    X_ = np.zeros((X.shape[0],0))
    for na in n_agg:
        if na == 0:
            XA = X.copy()
        else:
            XA = aggregateEmbedding(X, n_groups = na)
        X_ = np.concatenate((X_, XA), axis = 1)
    return X_

In [10]:
#Gs =  [(nx.from_numpy_array(nx.to_numpy_array(nx.karate_club_graph(), weight=None)), 'karate', 4)]#, (nx.les_miserables_graph(),'lesmis',6)]

In [11]:
#processNet('karate_34') #karate_34 & 0.419790 & 0.419790 & 0.419790 & 0.419790 & 0.05 & 0.12 & 0.14 & 1.25
#processNet('dolphins_62') #ok dolphins_62 & 0.527728 & 0.526463 & 0.527728 & 0.528519 & 0.11 & 0.15 & 0.10 & 1.13
#processNet('football_115') #ok football_115 & 0.605445 & 0.605445 & 0.605445 & 0.605445 & 0.22 & 0.26 & 0.34 & 2.89
#        processNet('polbooks_105') #ok polbooks_105 & 0.526967 & 0.527237 & 0.527237 & 0.527237 & 0.22 & 0.23 & 0.18 & 1.57
#        processNet('copperfield_112') #ok ccopperfield_112 & 0.303028 & 0.309556 & 0.310173 & 0.313301 & 0.29 & 0.33 & 0.22 & 1.99
#        processNet('email_1133')  # ok - undirected, unweighted email_1133 & 0.573062 & 0.582755 & 0.572484 & 0.578221 & 5.15 & 47.57 & 7.36 & 71.62
#        processNet('lesmis_77') #ok lesmis_77 & 0.566688 & 0.566688 & 0.566688 & 0.566688 & 0.12 & 0.16 & 0.24 & 1.94
#        processNet('celegansneural_297')  # directed celegansneural_297 & 0.503509 & 0.507642 & 0.505710 & 0.507642 & 16.43 & 1.19 & 1.46 & 8.61 | seed = 1: & 0.503509 & 0.507605 & 0.506235 & 0.507642 & 16.32 & 1.09 & 1.45 & 8.26
#        processNet('jazz_198')  # celegansmetabolic_198 & 0.445627 & 0.444787 & 0.445522 & 0.445627 & 6.28 & 0.35 & 0.83 & 5.50
#        processNet('USAir97_332')  # seed = 1; USAir97_332 & 0.207925 & 0.215244 & 0.217149 & 0.217751 & 20.27 & 1.08 & 1.72 & 9.29


In [189]:
#G_ = Gs[0]
#G = G_[0]
#n_comms = G_[2]

nname = 'lesmis_77' 
#nname = 'copperfield_112'
#nname = 'polbooks_105'
#nname = 'jazz_198' #0.443
G = loadNetworkMat(nname)
n_comms = 10

print("Network ", nname, '; communities:', n_comms)
#G = G_[0]
A = nx.to_numpy_array(G)
Q = modularity_matrix_np(A)

Network  lesmis_77 ; communities: 10


In [190]:
#partition aggregated embeddings
#cluststyle = 'GM' 
cluststyle = 'k-means'
seed = 12
X_ = produceAggEmbed(A, n_agg = [20, 8, 4], alpha = 0.85)
N = X_.shape[1]
partition, _ = clusterEmbeddings(X = X_, n_comms = n_comms, cluststyle = cluststyle, seed = seed)
Q = modularity_matrix_np(A)
(Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()
#modularity(modularity_matrix(A), np.array(rpr_cluster_labels))

0.541158536585366

In [12]:
class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNNLayer, self).__init__()
        self.weight1 = nn.Parameter(torch.randn(in_features, out_features)) # 0.5 * torch.eye(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(1, out_features)) # -0.5 * torch.ones(1, out_features))
        self.dropout = dropout
        #xavier initialization
        torch.nn.init.xavier_uniform(self.weight1)
        torch.nn.init.xavier_uniform(self.bias)
        

    def forward(self, input):
        v1 = torch.mm(input, self.weight1)
        output = v1 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNN_MLP(nn.Module):
    def __init__(self, in_features, out_features, hid_layers = 0, hidden_dim=8, dropout=0.0):
        #super(GNN_MLP, self).__init__()
        super().__init__()
        self.n_layers = hid_layers + 1
        self.hidden_dim = hidden_dim
        if self.n_layers > 1:
            layers = [GNNLayer(in_features, self.hidden_dim, dropout)]
        else:
            layers = [GNNLayer(in_features, out_features, dropout)]
        for _ in range(self.n_layers-2):
            layers.append(GNNLayer(self.hidden_dim, self.hidden_dim, dropout))
        #if self.n_layers > 1:
        layers.append(GNNLayer(self.hidden_dim, out_features, dropout = 0))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = nn.ReLU()(x)
        x = self.layers[-1](x)
        x = nn.Softmax(dim=1)(x)
        #x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        #x = torch.clamp(x, 0, 1)
        #x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

In [141]:
class VNNpartitioner (GNN_MLP):
    def __init__(self, A, X, out_features = 2, hid_layers = 1, hidden_dim = 12, dropout = 0.0):
        self.adj = torch.FloatTensor(A)
        self.Q = modularity_matrix(self.adj)
        self.features = torch.FloatTensor(X)
        super().__init__(self.features.shape[1], out_features = out_features, hid_layers = hid_layers, hidden_dim = hidden_dim, dropout = dropout)
         
    def fitSupervised(self, target_comms, n_epochs = 1000, lr = 0.005, SEED = 1, batchsize = 0):
        track_best = True
        C0 = torch.tensor(to_categorical(target_comms))
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        t_total = time.time()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(n_epochs):
            batchind = range(self.features.shape[0])
            if batchsize > 0:
                batchind = np.random.choice(batchind, batchsize)
            t_1run = time.time()
            optimizer.zero_grad()
            out_embed = self.forward(self.features[batchind, :])
            C = out_embed[:, :n_comms]
            loss = torch.mean(torch.square(torch.subtract(C,C0[batchind,:]))) #torch.nn.functional.binary_cross_entropy(C, C0)
            loss.backward()
            optimizer.step()
            
            if track_best:
                if batchsize > 0:
                    full_embed = self.forward(self.features)
                    C = full_embed[:, :n_comms]
                    full_loss = torch.mean(torch.square(torch.subtract(C,C0)))
                else:
                    full_loss = loss
            
                if epoch == 0 or full_loss < best_loss:
                    best_loss = full_loss #- torch.trace(Q)
                    best_C = C.data
                    best_embed = out_embed.data
                    best_epoch = epoch
                
            else:
                full_loss = loss
                best_loss = loss
                
            if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
                #optimizer = optim.Adam(model.parameters(), lr=lr)
                print('Epoch: {:04d}'.format(epoch + 1), 'batch MSE: {:.8f}'.format(loss.item()), 'full MSE: {:.8f}'.format(full_loss.item()),
                        'best MSE: {:.8f}'.format(best_loss.item()),
                        'time: {:.4f}s'.format(time.time() - t_1run))
        ent = best_loss.item()
        print("Optimization Finished with MSE ", ent)
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        return C, ent
    
    def fitUnsupervised(self, n_epochs = 1000, lr = 0.005, SEED = 1, batchsize = 0, restore_best = 500):
        track_best = True
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        t_total = time.time()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        
        out_embed = self.forward(self.features)
        C = out_embed[:, :n_comms].data
        
        for epoch in range(n_epochs):
            t_1run = time.time()
            optimizer.zero_grad()
            batchind = range(self.features.shape[0])
            if batchsize > 0:
                batchind = np.random.choice(batchind, batchsize)
            out_embed = self.forward(self.features[batchind, :])
            C[batchind, :] = out_embed[:, :n_comms]
            Q1 = torch.mm(C.T, self.Q)
            Q2 = torch.mm(Q1, C)
            loss = torch.trace(Q2)
            loss = -loss
            loss.backward()
            optimizer.step()
            C = C.data
            
            if track_best:
                if batchsize > 0:
                    full_embed = self.forward(self.features)
                    C = full_embed[:, :n_comms].data
                    full_loss = - torch.trace(torch.mm(torch.mm(C.T, self.Q), C))
                else:
                    full_loss = loss
            
                if epoch == 0 or full_loss < best_loss:
                    best_loss = full_loss #- torch.trace(Q)
                    best_C = C.data
                    best_embed = out_embed.data
                    best_epoch = epoch
                    
                    torch.save(self.state_dict(), 'model_scripted.pt')
                    
                if (epoch + 1) % restore_best == 0:
                    self.load_state_dict(torch.load('model_scripted.pt'))
                    self.eval()
                
            else:
                full_loss = loss
                best_loss = loss
            
            if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
                #optimizer = optim.Adam(model.parameters(), lr=lr)
                print('Epoch: {:04d}'.format(epoch + 1), 'modularity: batch: {:.8f}'.format(-loss.item()), 'full: {:.8f}'.format(-full_loss.item()),
                        'best: {:.8f}'.format(-best_loss.item()),
                        'time: {:.4f}s'.format(time.time() - t_1run))
                #print('Epoch: {:04d}'.format(epoch + 1),
                #        'Modularity: {:.8f}'.format(-best_loss.item()),
                #        'time: {:.4f}s'.format(time.time() - t_1run))
        mod = -best_loss.item()
        print("Optimization Finished with modularity ", mod)
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        return C, mod
        
        

In [19]:
#combo_comms, combo_mod = pycombo.execute(G, max_communities = 3); combo_mod

In [192]:
X_ = produceAggEmbed(A, n_agg = [20, 8, 4])

In [180]:

#augment X_ with cluster centroids
#partition, model = clusterEmbeddings(X = X_, n_comms = n_comms)
#XC = model.cluster_centers_
#XC = np.array((list(XC.flatten()) * X_.shape[0])).reshape(X_.shape[0],-1)
#X_ = np.concatenate((X_, XC), axis = 1)

In [193]:
partitioner = VNNpartitioner(A, X_, out_features = n_comms, hid_layers = 4, hidden_dim = 2*N, dropout = 0.1)

/Users/stanislav/opt/anaconda3/envs/Python37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/Users/stanislav/opt/anaconda3/envs/Python37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':


In [194]:
partitioner.fitSupervised(partition, n_epochs = 5000, lr = 0.0005, SEED = 2, batchsize = 20);

Epoch: 0001 batch MSE: 0.09287868 full MSE: 0.09311878 best MSE: 0.09311878 time: 0.0029s
Epoch: 0251 batch MSE: 0.04260942 full MSE: 0.04416693 best MSE: 0.04307650 time: 0.0022s
Epoch: 0501 batch MSE: 0.00362893 full MSE: 0.01388867 best MSE: 0.01326894 time: 0.0024s
Epoch: 0751 batch MSE: 0.01411554 full MSE: 0.01105171 best MSE: 0.00828037 time: 0.0022s
Epoch: 1001 batch MSE: 0.00530259 full MSE: 0.01071591 best MSE: 0.00681864 time: 0.0022s
Epoch: 1251 batch MSE: 0.00700580 full MSE: 0.00808174 best MSE: 0.00563335 time: 0.0025s
Epoch: 1501 batch MSE: 0.00798272 full MSE: 0.00623938 best MSE: 0.00534532 time: 0.0023s
Epoch: 1751 batch MSE: 0.00109555 full MSE: 0.00323535 best MSE: 0.00240318 time: 0.0021s
Epoch: 2001 batch MSE: 0.01078378 full MSE: 0.00202744 best MSE: 0.00127370 time: 0.0021s
Epoch: 2251 batch MSE: 0.00003505 full MSE: 0.00192861 best MSE: 0.00034608 time: 0.0030s
Epoch: 2501 batch MSE: 0.00529000 full MSE: 0.00082043 best MSE: 0.00016680 time: 0.0026s
Epoch: 275

In [196]:
partitioner.dropout = 0.0

In [198]:
partitioner.fitUnsupervised(n_epochs = 10000, lr = 0.0001, SEED = 1, batchsize = 20, restore_best = 200);

Epoch: 0001 modularity: batch: 0.54844773 full: 0.54844767 best: 0.54844767 time: 0.0036s
Epoch: 0501 modularity: batch: 0.54844737 full: 0.54844707 best: 0.54844773 time: 0.0018s
Epoch: 1001 modularity: batch: 0.54844773 full: 0.54844761 best: 0.54844773 time: 0.0018s
Epoch: 1501 modularity: batch: 0.56502473 full: 0.56502473 best: 0.56502473 time: 0.0017s
Epoch: 2001 modularity: batch: 0.56503075 full: 0.56503075 best: 0.56503075 time: 0.0017s
Epoch: 2501 modularity: batch: 0.56503189 full: 0.56503189 best: 0.56503189 time: 0.0018s
Epoch: 3001 modularity: batch: 0.56503248 full: 0.56503248 best: 0.56503248 time: 0.0017s
Epoch: 3501 modularity: batch: 0.56503278 full: 0.56503278 best: 0.56503278 time: 0.0018s
Epoch: 4001 modularity: batch: 0.56503302 full: 0.56503302 best: 0.56503302 time: 0.0018s
Epoch: 4501 modularity: batch: 0.56503308 full: 0.56503308 best: 0.56503308 time: 0.0018s
Epoch: 5001 modularity: batch: 0.56503314 full: 0.56503308 best: 0.56503314 time: 0.0022s
Epoch: 550